In [1]:
import os
import torch
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

# Get number of cores
print(os.cpu_count())

# Get number of threads
print(torch.get_num_threads())

True
Tesla V100-SXM2-32GB
34072559616
72
36


In [2]:
from model import RoadModel
from pytorch_lightning.loggers import WandbLogger

from src import RoadDataModule, LogPredictionsCallback, val_checkpoint, regular_checkpoint


def main(cfg: DictConfig) -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RoadModel(cfg, device)
    datamodule = RoadDataModule(cfg)

    wandb_logger = WandbLogger(project="road-segmentation", name="baseline", log_model='all')

    trainer = L.Trainer(max_epochs=5, accelerator="gpu", devices=1, logger=wandb_logger, callbacks=[
        LogPredictionsCallback(),
        val_checkpoint,
        regular_checkpoint
    ])
    trainer.fit(model, datamodule=datamodule)
    # trainer.test(model, datamodule=datamodule)
    # trainer.predict(model, datamodule=datamodule)

In [3]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf"):
    config = compose(config_name="config")
    main(config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aleskucera. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | DeepLabV3              | 39.6 M
1 | criterion | CrossEntropyLoss       | 0     
2 | accuracy  | MulticlassAccuracy     | 0     
3 | jaccard   | MulticlassJaccardIndex | 0     
-----------------------------------------------------
39.6 M    Trainable params
0         Non-trainable params
39.6 M    Total params
158.538   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

MisconfigurationException: `ModelCheckpoint(monitor='valid_jaccard')` could not find the monitored key in the returned metrics: ['train_loss', 'train_loss_step', 'val_loss', 'val_loss_epoch', 'val_accuracy', 'val_jaccard', 'train_loss_epoch', 'train_accuracy', 'train_jaccard', 'epoch', 'step']. HINT: Did you call `log('valid_jaccard', value)` in the `LightningModule`?